# Sequence models

Finally, we arrive a sequence models. Hopefully this will serve us better than the convolutional approach (but why would it?).

## 1. RNN for input-output with equal length
```
Input: pianoroll-unit (128, 96)
Output: pianoroll-unit (128, 96)
```

Code adapted from: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [1]:
import os, shutil
import random
import sys
import numpy as np
import pypianoroll
from matplotlib import pyplot as plt
import cPickle as pickle
import pianoroll_utils

from keras.layers import Input, Dense, Conv2D, Conv2DTranspose, BatchNormalization, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.models import load_model

PICKLE_FILE = './pickle_jar/norm_units_50_songs.pkl'

Using TensorFlow backend.


In [2]:
units = {}
with open(PICKLE_FILE, 'rb') as infile:
    units = pickle.load( infile )
units['full'] = units['input'] + units['comp']

# Print info
print("Loaded", units["input"].shape[0], "units from", PICKLE_FILE)
print("input_units.shape: ", units["input"].shape)
print("comp_units.shape: ", units["comp"].shape)
print("full_units.shape: ", units["full"].shape)

# Create an array of True (train) and False (test) to split the dataset
train_test_indices = np.random.choice([True, False], size=len(units["input"]), p=[.9, .1])
# Training data
input_train = units["input"][train_test_indices, ...].swapaxes(1,2)
comp_train = units["comp"][train_test_indices, ...].swapaxes(1,2)
full_train = units["full"][train_test_indices, ...].swapaxes(1,2)
# Testing data
input_test = units["input"][np.invert(train_test_indices), ...].swapaxes(1,2)
comp_test = units["comp"][np.invert(train_test_indices), ...].swapaxes(1,2)
full_test = units["full"][np.invert(train_test_indices), ...].swapaxes(1,2)
print("Train:", input_train.shape)
print("Test:", input_test.shape)

('Loaded', 3831, 'units from', './pickle_jar/norm_units_50_songs.pkl')
('input_units.shape: ', (3831, 128, 96))
('comp_units.shape: ', (3831, 128, 96))
('full_units.shape: ', (3831, 128, 96))
('Train:', (3433, 96, 128))
('Test:', (398, 96, 128))


In [3]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
NUM_TICKS = 96
NUM_PITCHES = 128
REVERSE = True

MAXLEN = NUM_TICKS

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, NUM_PITCHES)))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(MAXLEN))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(NUM_PITCHES)))
model.add(layers.Activation('relu'))
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 96, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 96, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 96, 128)           16512     
_________________________________________________________________
activation_1 (Activation)    (None, 96, 128)           0         
Total params: 279,680
Trainable params: 279,680
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Train the model each generation and show predictions against the validation
# dataset.

model.fit(input_train, input_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(input_test, input_test),
                callbacks=[TensorBoard(log_dir='/tmp/rnn')])

MODEL_RNN_V1_FILE = './models/rnn_v1.h5'
model.save(MODEL_RNN_V1_FILE)# creates a HDF5 file
print("Saved Keras model to", MODEL_RNN_V1_FILE)

Instructions for updating:
Use the retry module or similar alternatives.
Train on 3433 samples, validate on 398 samples
Epoch 1/50
3433/3433 [==============================] - 30s 9ms/step - loss: 0.0058 - acc: 0.2666 - val_loss: 0.0049 - val_acc: 0.3286
Epoch 2/50
3433/3433 [==============================] - 29s 8ms/step - loss: 0.0044 - acc: 0.3638 - val_loss: 0.0044 - val_acc: 0.3616
Epoch 3/50
3433/3433 [==============================] - 29s 8ms/step - loss: 0.0041 - acc: 0.3969 - val_loss: 0.0042 - val_acc: 0.3625
Epoch 4/50
3433/3433 [==============================] - 29s 8ms/step - loss: 0.0039 - acc: 0.4053 - val_loss: 0.0041 - val_acc: 0.4004
Epoch 5/50
3433/3433 [==============================] - 29s 8ms/step - loss: 0.0037 - acc: 0.4117 - val_loss: 0.0039 - val_acc: 0.4092
Epoch 6/50
3433/3433 [==============================] - 29s 8ms/step - loss: 0.0037 - acc: 0.4179 - val_loss: 0.0039 - val_acc: 0.4028
Epoch 7/50
3433/3433 [==============================] - 29s 8ms/step -

NameError: name 'autoencoder' is not defined